In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import qc, config, tools

from importlib import reload

pd.options.display.max_rows = 9999
pd.options.display.max_columns = 9999

In [2]:
reload(qc)

#query_n = 75

a = qc.QueryQC()
a.test = "ds"
a.scale = 1
a.cid = "01"
a.stream_n = 1
a.desc =  f"qc_query"
a.seq_id = "NA"

a.verbose = False
a.verbose_query = True
a.qual = False
a.save = True
a.verbose_iter = False

a.set_timestamp_dir()

#a.run_single(query_n = query_n)

seq = tools.tpc_stream(test="ds", n=1)
#seq = tools.tpc_stream(test="h", n=1)

#seq = [31, 36, 51, 70, 75, 86, 98]

# 31 sort order in Pandas fixed

# 36 grouping removed, total results not limited to 100, 115 now return, 71% identical

# 70 grouping, rollup not supported by BQ both removed

# 75 UNION / UNION ALL / UNION DISTINCT - BQ changed from UNION to UNION DISTINCT

# 86 grouping > removed grouping, queries are not 59% identical

# errors this pass
# [27, 36, 49, 51, 86, 98]

# 27 grouping removed

# 49 99.9% complete, one value off - this is a floating point difference!

# 51 row 22 is off, this row: 22	22	1998-02-06	-9999	65	65	65

# 98 row 1157 is off revenueratio is 0 in SF and 1 in BQ

# BQ data:
# AAAAAAAACNPBAAAA,Single points see worst stones. Sometimes large ra,Music,country,4.880,261.500,0.022

# SF data:
# AAAAAAAACNPBAAAA,Single points see worst stones. Sometimes large ra,Music,country,4.880,261.500,0.02198519


seq = [98]

a.run(seq=seq)
df_results = a.compare()

Result Folder Name:
/home/colin/code/bq_snowflake_benchmark/results/bqsf_ds_01_results--qc_query-2020-06-12_21:09:07.153704
QUERY TEXT
0  ALTER SESSION SET TIMEZONE = 'UTC'

QUERY TEXT
0  ALTER SESSION SET QUERY_TAG = 'ds_1gb_01-xx-qc_query'

QUERY TEXT
0  ALTER SESSION SET USE_CACHED_RESULT=false

QUERY TEXT
0  USE WAREHOUSE TEST9000

QUERY TEXT
0  USE DATABASE ds_1GB_01

QUERY TEXT
0  ALTER SESSION SET QUERY_TAG = 'ds_1gb_01-q98-na-qc_query'

QUERY TEXT
0  
1  select i_item_id
2        ,i_item_desc 
3        ,i_category 
4        ,i_class 
5        ,i_current_price
6        ,sum(ss_ext_sales_price) as itemrevenue 
7        ,sum(ss_ext_sales_price)*100/sum(sum(ss_ext_sales_price)) over
8            (partition by i_class) as revenueratio
9  from	
10  	store_sales
11      	,item 
12      	,date_dim
13  where 
14  	ss_item_sk = i_item_sk 
15    	and i_category in ('Shoes', 'Music', 'Children')
16    	and ss_sold_date_sk = d_date_sk
17  	and d_date between to_date('1998-01-04')
18  	     

In [3]:
x = df_results.match.sum()/len(df_results)
print(f"Complete: {x*100}%")
print("Error Queries:",
      list(df_results.loc[df_results.match == False, "q_sf"].values))

Complete: 0.0%
Error Queries: [98]


In [4]:
df_sf = a.result_sf
df_bq = a.result_bq
df_sf1 = a.result_sf_csv
df_bq1 = a.result_sf_csv

In [5]:
df_sf.dtypes

I_ITEM_ID           object
I_ITEM_DESC         object
I_CATEGORY          object
I_CLASS             object
I_CURRENT_PRICE    float64
ITEMREVENUE        float64
REVENUERATIO        object
dtype: object

In [6]:
df_bq.dtypes

i_item_id           object
i_item_desc         object
i_category          object
i_class             object
i_current_price    float64
itemrevenue        float64
revenueratio       float64
dtype: object

In [7]:
df_sf.columns = [s.lower() for s in df_sf.columns]

In [8]:
df_bq.head(10)

,i_item_id,i_item_desc,i_category,i_class,i_current_price,itemrevenue,revenueratio
0,AAAAAAAAABECAAAA,So equal implications would not move closely f...,Children,infants,1.06,148.93,0.011991
1,AAAAAAAAABFCAAAA,Also satisfactory values may cover all. Nation...,Children,infants,4.98,7252.16,0.583907
2,AAAAAAAAABKDAAAA,"Statements must not find polish, economic hole...",Children,infants,2.56,10505.35,0.845837
3,AAAAAAAAACCBAAAA,Certain consequences belong very things; actua...,Children,infants,3.72,22975.11,1.849838
4,AAAAAAAAACOBAAAA,Complex farmers shall attend in a homes.,Children,infants,2.57,17193.93,1.384367
5,AAAAAAAAADHDAAAA,Also practical weeks ask on a circumstances. S...,Children,infants,2.07,4421.48,0.355995
6,AAAAAAAAADNAAAAA,Ra,Children,infants,2.80,5499.42,0.442785
7,AAAAAAAAADOAAAAA,Other aspects offer then however m,Children,infants,6.46,7418.60,0.597308
8,AAAAAAAAADOCAAAA,"National, clear liabilities cry in a hours. Ju...",Children,infants,0.95,9498.41,0.764763
9,AAAAAAAAAEJAAAAA,Eyes can split also grounds. Comparable users ...,Children,infants,2.07,7083.33,0.570313


In [9]:
df_sf.head(10)

,i_item_id,i_item_desc,i_category,i_class,i_current_price,itemrevenue,revenueratio
0,AAAAAAAAABECAAAA,So equal implications would not move closely f...,Children,infants,1.06,148.93,0.01199108
1,AAAAAAAAABFCAAAA,Also satisfactory values may cover all. Nation...,Children,infants,4.98,7252.16,0.58390670
2,AAAAAAAAABKDAAAA,"Statements must not find polish, economic hole...",Children,infants,2.56,10505.35,0.84583686
3,AAAAAAAAACCBAAAA,Certain consequences belong very things; actua...,Children,infants,3.72,22975.11,1.84983792
4,AAAAAAAAACOBAAAA,Complex farmers shall attend in a homes.,Children,infants,2.57,17193.93,1.38436698
5,AAAAAAAAADHDAAAA,Also practical weeks ask on a circumstances. S...,Children,infants,2.07,4421.48,0.35599487
6,AAAAAAAAADNAAAAA,Ra,Children,infants,2.80,5499.42,0.44278507
7,AAAAAAAAADOAAAAA,Other aspects offer then however m,Children,infants,6.46,7418.60,0.59730759
8,AAAAAAAAADOCAAAA,"National, clear liabilities cry in a hours. Ju...",Children,infants,0.95,9498.41,0.76476322
9,AAAAAAAAAEJAAAAA,Eyes can split also grounds. Comparable users ...,Children,infants,2.07,7083.33,0.57031337


In [10]:
df_bq.tail(10)

,i_item_id,i_item_desc,i_category,i_class,i_current_price,itemrevenue,revenueratio
2555,AAAAAAAAPFLDAAAA,Sometimes used workers may ask bodies. Areas m...,Shoes,womens,8.78,14505.20,1.268371
2556,AAAAAAAAPHKCAAAA,Dual millions enjoy with a,Shoes,womens,3.34,1180.06,0.103187
2557,AAAAAAAAPIBBAAAA,Etc new patients occur rapidly more educationa...,Shoes,womens,3.25,10741.18,0.939235
2558,AAAAAAAAPIIAAAAA,"Public conservatives take all rational, light ...",Shoes,womens,1.69,7357.95,0.643397
2559,AAAAAAAAPJNDAAAA,Very other wages will not call groups. Public ...,Shoes,womens,6.32,18247.61,1.595616
2560,AAAAAAAAPLLDAAAA,Then financial days choose most there new deta...,Shoes,womens,7.34,12663.32,1.107312
2561,AAAAAAAAPMKAAAAA,Teachers become hills. Over typical pri,Shoes,womens,33.72,2299.87,0.201106
2562,AAAAAAAAPONBAAAA,Free lands should not expect too personal offi...,Shoes,womens,9.61,3638.03,0.318118
2563,AAAAAAAAPPBAAAAA,New concentrations produce. Too technical minu...,Shoes,womens,1.31,7272.48,0.635924
2564,AAAAAAAAPPHDAAAA,Responsible cases turn that is. Unlikely chara...,Shoes,womens,2.21,1351.61,0.118188


In [11]:
df_sf.tail(10)

,i_item_id,i_item_desc,i_category,i_class,i_current_price,itemrevenue,revenueratio
2555,AAAAAAAAPFLDAAAA,Sometimes used workers may ask bodies. Areas m...,Shoes,womens,8.78,14505.20,1.26837059
2556,AAAAAAAAPHKCAAAA,Dual millions enjoy with a,Shoes,womens,3.34,1180.06,0.10318737
2557,AAAAAAAAPIBBAAAA,Etc new patients occur rapidly more educationa...,Shoes,womens,3.25,10741.18,0.93923537
2558,AAAAAAAAPIIAAAAA,"Public conservatives take all rational, light ...",Shoes,womens,1.69,7357.95,0.64339736
2559,AAAAAAAAPJNDAAAA,Very other wages will not call groups. Public ...,Shoes,womens,6.32,18247.61,1.59561619
2560,AAAAAAAAPLLDAAAA,Then financial days choose most there new deta...,Shoes,womens,7.34,12663.32,1.10731205
2561,AAAAAAAAPMKAAAAA,Teachers become hills. Over typical pri,Shoes,womens,33.72,2299.87,0.20110633
2562,AAAAAAAAPONBAAAA,Free lands should not expect too personal offi...,Shoes,womens,9.61,3638.03,0.31811835
2563,AAAAAAAAPPBAAAAA,New concentrations produce. Too technical minu...,Shoes,womens,1.31,7272.48,0.63592365
2564,AAAAAAAAPPHDAAAA,Responsible cases turn that is. Unlikely chara...,Shoes,womens,2.21,1351.61,0.11818812


In [16]:
def color_false(val):
    mapper = {True: 'white', False: 'red'}
    return 'color: %s' % mapper[val]

In [43]:
diff = df_sf.eq(df_bq)
diff.loc[0:5].style.applymap(color_false)

,i_item_id,i_item_desc,i_category,i_class,i_current_price,itemrevenue,revenueratio
0,True,True,True,True,True,True,False
1,True,True,True,True,True,True,False
2,True,True,True,True,True,True,False
3,True,True,True,True,True,True,False
4,True,True,True,True,True,True,False
5,True,True,True,True,True,False,False


In [18]:
diff.sum().sum() / (diff.shape[0] * diff.shape[1])

0.8250069618490671

In [24]:
df_sf.loc[1157, "itemrevenue"]

14067.99

In [21]:
df_bq.loc[1157]

i_item_id                                           AAAAAAAADBFEAAAA
i_item_desc        Mean, old months eat less local criteria. Also...
i_category                                                     Music
i_class                                                      country
i_current_price                                                 1.16
itemrevenue                                                    14068
revenueratio                                                 1.18274
Name: 1157, dtype: object

In [27]:
x = df_sf.loc[5, "itemrevenue"]
x

4421.48

In [28]:
y = df_bq.loc[5, "itemrevenue"]
y

4421.4800000000005

In [29]:
x == y

False

In [30]:
type(x)

numpy.float64

In [31]:
type(y)

numpy.float64

In [32]:
g = df_sf.loc[5, "revenueratio"]
g

Decimal('0.35599487')

In [33]:
h = df_bq.loc[5, "revenueratio"]
h

0.3559948732963709

In [34]:
str(h)

'0.3559948732963709'

In [35]:
import decimal

In [39]:
decimal.Decimal(str(h)[:5])

Decimal('0.355')